# LUME-services demo
In this notebook, we will configure LUME-services to use the service configuration used to launch our docker-compose services. Make sure you've completed all steps outlined in https://slaclab.github.io/lume-services/demo/.

In [8]:
import logging
logging.basicConfig(level=logging.INFO)  # Lets check the logs

## Configure services
LUME-services is packages with a configuration utility that reads environment variables and initializes services:

In [9]:
from lume_services import config
config.configure()

INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [10]:
#model_db_service._reset()

## Create a model
The LUME-services Model provides an API to all model services and facilitates all model operations.

In [11]:
from lume_services.models import Model

model = Model.create_model(
    author = "Aman Singh Thakur",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu",
    description = "lcls-cu-inj-nn"
)
model

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.author = :author_1 AND model.laboratory = :laboratory_1 AND model.facility = :facility_1 AND model.beampath = :beampath_1 AND model.description = :description_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2023, 8, 1, 2, 11, 10),                     author='Aman Singh Thakur'),                     laboratory='slac',                     facility='lcls',                     beampath='cu',                     description='lcls-cu-inj-nn'                 ), deployment=None, results=None)

## Create a project
Workflows are organized by the Prefect scheduler into different projects. Below, we access the configured services directly (TODO create project registry utility)

In [12]:
model_db_service = config.context.model_db_service()
scheduling_service = config.context.scheduling_service()

In [13]:
### NOTE: The below cell will raise an error if run 2x

In [14]:
# create a project
project_name = model_db_service.store_project(
    project_name="gpu-box-new2", description="my_description"
)
scheduling_service.create_project("gpu-box-new2")

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


You can now find this project in you Prefect UI at http://localhost:8080


![project](https://slaclab.github.io/lume-services/files/project_nav.png)

## Create a deployment for your model
Replace `source_path` with the path to your release tarball below:

In [15]:
#source_path = "https://github.com/jacquelinegarrahan/lume-lcls-cu-inj-nn/releases/download/v0.0.12/lume_lcls_cu_inj_nn-0.0.12.tar.gz"

source_path = "https://github.com/singh96aman/lume-lcls-cu-inj-nn/releases/download/v0.0.20/lume_lcls_cu_inj_nn-0.0.20.tar.gz"

# populates local channel
model.store_deployment(source_path, project_name="gpu-box-new2")
model

INFO:lume_services.models.model:installing package
INFO:lume_services.environment.solver:https://github.com/singh96aman/lume-lcls-cu-inj-nn/releases/download/v0.0.20/lume_lcls_cu_inj_nn-0.0.20.tar.gz saved to /tmp/tmpw0b2t60e/lume_lcls_cu_inj_nn-0.0.20.tar.gz
INFO:lume_services.environment.solver:Version 0.0.20 of lume_lcls_cu_inj_nn already installed.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.sha256, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 AND deployment.version = :version_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow.flow_id, flow.flow_name, flow.project_name, flow.deployment_id 
FROM flow 
WHERE flow.deployment_id = :deployment_id_1
INFO:lume_services.services.scheduling.backends.server:Flow run config is

Flow URL: http://localhost:8080/default/flow/8b084a95-27d1-4c9f-ab78-44b4ff31a1ae
 └── ID: 7f896d32-d0b1-4a25-9f3b-9a4afe1d5301
 └── Project: gpu-box-new2
 └── Labels: ['lume-services']


Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2023, 8, 1, 2, 11, 10),                     author='Aman Singh Thakur'),                     laboratory='slac',                     facility='lcls',                     beampath='cu',                     description='lcls-cu-inj-nn'                 ), deployment=Deployment(metadata=Deployment(                 deployment_id=1,                 model_id=1,                 version='0.0.20',                 deploy_date=datetime.datetime(2023, 8, 1, 2, 11, 56)),                 asset_dir=None,                 source='https://github.com/singh96aman/lume-lcls-cu-inj-nn/releases/download/v0.0.20/lume_lcls_cu_inj_nn-0.0.20.tar.gz',                 sha256='ddfa081fef58a639fe132ef02327a3eb98556e9bea4478ce38c211c414b60153',                 image='scr.svc.stanford.edu/aman96/lume-lcls-cu-inj-nn:v0.0.20',                 is_live=True                 package_import_name='lume_lcls_cu_inj_nn'            

## Run the Prefect workflow directly

In [16]:
flow_run = model.deployment.flow.prefect_flow.run(**{
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    }
)


[2023-07-31 19:12:29-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'lume-lcls-cu-inj-nn'


INFO:prefect.FlowRunner:Beginning Flow run for 'lume-lcls-cu-inj-nn'


[2023-07-31 19:12:29-0700] INFO - prefect.TaskRunner | Task 'configure_lume_services': Starting task run...


INFO:prefect.TaskRunner:Task 'configure_lume_services': Starting task run...
INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


[2023-07-31 19:12:29-0700] INFO - prefect.TaskRunner | Task 'configure_lume_services': Finished task run for task with final state: 'Success'


...

INFO:prefect.TaskRunner:Task 'save_db_result': Starting task run...


[2023-07-31 19:12:31-0700] INFO - prefect.TaskRunner | Task 'save_db_result': Finished task run for task with final state: 'TriggerFailed'


INFO:prefect.TaskRunner:Task 'save_db_result': Finished task run for task with final state: 'TriggerFailed'


[2023-07-31 19:12:31-0700] INFO - prefect.FlowRunner | Flow run FAILED: some reference tasks failed.


INFO:prefect.FlowRunner:Flow run FAILED: some reference tasks failed.


## Run the workflow inside the service cluster
We can use the model interface to directly deploy workflows. When sourcing our environment (`docs/examples/demo.env`), we defined a mount point for the file system using the alias `/lume-services/data`. Let's kick off this workflow and save the file output to that directory. 
After running the next cell, you'll be able to see the running container in your docker desktop and examine the flow using the Prefect UI at http://localhost:8080/default?flows.

In [17]:
model.run(
    parameters = {
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    }
)

In [18]:
model.deployment.flow

Flow(name='lume-lcls-cu-inj-nn', flow_id='7f896d32-d0b1-4a25-9f3b-9a4afe1d5301', project_name='gpu-box-new2', prefect_flow=<Flow: name="lume-lcls-cu-inj-nn">, parameters=None, mapped_parameters=None, task_slugs=None, labels=['lume-services'], image='scr.svc.stanford.edu/aman96/lume-lcls-cu-inj-nn:v0.0.20')

# Get results:

In [ ]:
results = model.get_results()
results

In [ ]:
results_df = model.get_results_df()
results_df

## Load model using model id
Once your model has been registered, you can use the `Model` api object to interact with your model without running the above registration steps. Let's load a new model object using the model_id we registered above.

In [3]:
from importlib import reload 
import lume_services
reload(lume_services.models)
from lume_services.models import Model

#model_id = model.metadata.model_id
loaded_model = Model(model_id=1)

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


In [4]:
loaded_model.metadata

Model(                     model_id=1,                     created=datetime.datetime(2023, 8, 1, 0, 47, 35),                     author='Aman Singh Thakur'),                     laboratory='slac',                     facility='lcls',                     beampath='cu',                     description='lcls-cu-inj-nn'                 )

## Load existing model object
Loading a model using the load_deployment method without passing a deployment_id will load the latest deployment registered for the model.

In [5]:
loaded_model.load_deployment()
loaded_model.deployment

INFO:lume_services.models.model:Loading latest deployment.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.sha256, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 ORDER BY deployment.deploy_date DESC
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow.flow_id, flow.flow_name, flow.project_name, flow.deployment_id 
FROM flow 
WHERE flow.deployment_id = :deployment_id_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT project.project_name, project.description 
FROM project 
WHERE project.project_name = :project_name_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow_of_flows._id, flow_of_flows.parent_flow_id, flow_of_flows.flow_id, flow_of_flows.position 
FROM flow_of_flows 
WHERE flow_of_fl

Deployment(metadata=Deployment(                 deployment_id=1,                 model_id=1,                 version='0.0.20',                 deploy_date=datetime.datetime(2023, 8, 1, 0, 49, 15)),                 asset_dir=None,                 source='https://github.com/singh96aman/lume-lcls-cu-inj-nn/releases/download/v0.0.20/lume_lcls_cu_inj_nn-0.0.20.tar.gz',                 sha256='ddfa081fef58a639fe132ef02327a3eb98556e9bea4478ce38c211c414b60153',                 image='scr.svc.stanford.edu/aman96/lume-lcls-cu-inj-nn:v0.0.20',                 is_live=True                 package_import_name='lume_lcls_cu_inj_nn'                 ), project=Project(metadata=Project(                 project_name='gpu-box-new',                 description='my_description',                 )), flow=Flow(name='lume-lcls-cu-inj-nn', flow_id='a4712eac-e30a-4abc-b6e9-56e0c706be8e', project_name='gpu-box-new', prefect_flow=None, parameters=None, mapped_parameters=None, task_slugs=None, labels=['lume-servic

In [ ]:
result = loaded_model.run_and_return(
    parameters={
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    },
    task_name="save_db_result" # Want to get the result from the save_db_result task
)
result

In [ ]:
results_df = loaded_model.get_results_df()
results_df